### Логистическая регрессия

### 1. Оцените качество логистической регрессии (sklearn.linear_model.LogisticRegression с L2-регуляризацией) с помощью кросс-валидации по той же схеме, которая использовалась для градиентного бустинга. 
Подберите при этом лучший параметр регуляризации (C). Какое наилучшее качество у вас получилось? Как оно соотносится с качеством градиентного бустинга? Чем вы можете объяснить эту разницу? Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?

In [2]:
import json
import bz2
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import datetime
import time

In [3]:
features = pd.read_csv('./data/features.csv', index_col='match_id')
features = features.drop(['duration', 'tower_status_radiant', 'tower_status_dire', 'barracks_status_radiant', 'barracks_status_dire'], axis=1)
y = features['radiant_win']
features = features.drop(['radiant_win'], axis=1)
features = features.fillna(0)
s = features.count()
s[s != 97230]

Series([], dtype: int64)

In [4]:
features.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,0,35.0,103.0,-84.0,221.0,3,4,2,2,-52.0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,0,-20.0,149.0,-84.0,195.0,5,4,3,1,-5.0
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,1,-39.0,45.0,-77.0,221.0,3,4,3,1,13.0
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,0,-30.0,124.0,-80.0,184.0,0,4,2,0,27.0
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,0,46.0,182.0,-80.0,225.0,6,3,3,0,-16.0


In [5]:
# clf = LogisticRegression(penalty='l2', random_state=241)
clf = LogisticRegression(penalty='l2')
# cv = KFold(n_splits=5, shuffle=True, random_state=241)
cv = KFold(n_splits=5, shuffle=True)
grid = {
    'C': np.arange(.5, 10.5, .5),
}
gs1 = GridSearchCV(clf, grid, scoring='roc_auc', cv=cv, n_jobs=-1)

In [6]:
start_time1 = datetime.datetime.now()
gs1.fit(features, y)
end_time1 = datetime.datetime.now()

/home/worker/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [7]:
gs1.best_estimator_

LogisticRegression(C=0.5, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [8]:
end_time1 - start_time1

datetime.timedelta(seconds=23, microseconds=17509)

In [9]:
for k, v in gs1.cv_results_.items():
    if 'mean' in k:
        print(f"Key: {k}, value: {v}\n")

Key: mean_fit_time, value: [0.6727859  0.67550139 0.66678667 0.67375736 0.67984567 0.67422662
 0.68067012 0.68644557 0.67693596 0.68130765 0.67726512 0.6743928
 0.66796947 0.66752491 0.67348986 0.6725987  0.67204189 0.67960114
 0.66796088 0.59517274]

Key: mean_score_time, value: [0.04996352 0.02876706 0.02534966 0.0287756  0.02681818 0.02616401
 0.02722573 0.02478061 0.0251853  0.02692885 0.02582183 0.02588372
 0.02418151 0.02507811 0.02386699 0.02468491 0.02662635 0.02650256
 0.02703342 0.02124624]

Key: mean_test_score, value: [0.51348033 0.51348033 0.51348033 0.51348033 0.51348033 0.51348033
 0.51348033 0.51348033 0.51348033 0.51348033 0.51348033 0.51348033
 0.51348033 0.51348033 0.51348033 0.51348033 0.51348033 0.51348033
 0.51348033 0.51348033]

Key: mean_train_score, value: [0.51345413 0.51345413 0.51345413 0.51345413 0.51345413 0.51345413
 0.51345413 0.51345413 0.51345413 0.51345413 0.51345413 0.51345413
 0.51345413 0.51345413 0.51345413 0.51345413 0.51345413 0.51345413
 0.5134

### Среди признаков в выборке есть категориальные, которые мы использовали как числовые, что вряд ли является хорошей идеей. 
Категориальных признаков в этой задаче одиннадцать: lobby_type и r1_hero, r2_hero, ..., r5_hero, d1_hero, d2_hero, ..., d5_hero. Уберите их из выборки, и проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. Изменилось ли качество? Чем вы можете это объяснить?

In [10]:
features_int = features.drop(['lobby_type', 
                              'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', 
                              'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero'],
                            axis=1)
clf = LogisticRegression(penalty='l2', random_state=241)
cv = KFold(n_splits=5, shuffle=True, random_state=241)
grid = {
    'C': np.arange(.5, 10.5, .5),
}
gs2 = GridSearchCV(clf, grid, scoring='roc_auc', cv=cv, n_jobs=-1)

In [11]:
start_time2 = datetime.datetime.now()
gs2.fit(features, y)
end_time2 = datetime.datetime.now()

/home/worker/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


start_time2 = datetime.datetime.now()
gs2.fit(features, y)
end_time2 = datetime.datetime.now()

In [12]:
gs2.best_estimator_

LogisticRegression(C=0.5, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=241, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [13]:
end_time2 - start_time2

datetime.timedelta(seconds=21, microseconds=631819)

In [14]:
for k, v in gs2.cv_results_.items():
    if 'mean' in k:
        print(f"Key: {k}, value: {v}\n")

Key: mean_fit_time, value: [0.6730495  0.67094049 0.67209697 0.66739721 0.66568308 0.66926394
 0.66553059 0.66616817 0.66437416 0.66479654 0.67319355 0.67629061
 0.67603688 0.67902093 0.68647156 0.68690796 0.68040323 0.67020717
 0.67363305 0.63517542]

Key: mean_score_time, value: [0.02751532 0.02583866 0.02438703 0.02728605 0.02665591 0.02574568
 0.02468171 0.02635522 0.02713161 0.02733517 0.0283402  0.02705917
 0.02726426 0.02576327 0.02515917 0.02405286 0.02605104 0.02687802
 0.02519751 0.02040577]

Key: mean_test_score, value: [0.51345564 0.51345564 0.51345564 0.51345564 0.51345564 0.51345564
 0.51345564 0.51345564 0.51345564 0.51345564 0.51345564 0.51345564
 0.51345564 0.51345564 0.51345564 0.51345564 0.51345564 0.51345564
 0.51345564 0.51345564]

Key: mean_train_score, value: [0.51345259 0.51345259 0.51345259 0.51345259 0.51345259 0.51345259
 0.51345259 0.51345259 0.51345259 0.51345259 0.51345259 0.51345259
 0.51345259 0.51345259 0.51345259 0.51345259 0.51345259 0.51345259
 0.513

### 3. На предыдущем шаге мы исключили из выборки признаки rM_hero и dM_hero, которые показывают, какие именно герои играли за каждую команду. 
Это важные признаки — герои имеют разные характеристики, и некоторые из них выигрывают чаще, чем другие. Выясните из данных, сколько различных идентификаторов героев существует в данной игре (вам может пригодиться фукнция unique или value_counts).

heroes = pd.read_csv('./data/dictionaries/heroes.csv')
heroes.count()  # Кол-во героев

In [16]:
heroes.head()

,id,localized_name,name
0,1,Anti-Mage,antimage
1,2,Axe,axe
2,3,Bane,bane
3,4,Bloodseeker,bloodseeker
4,5,Crystal Maiden,crystal_maiden


### 4. Какое получилось качество при добавлении "мешка слов" по героям? 
Улучшилось ли оно по сравнению с предыдущим вариантом? Чем вы можете это объяснить?

In [23]:
features = pd.read_csv('./data/features.csv', index_col='match_id')
features = features.drop(['duration', 'tower_status_radiant', 'tower_status_dire', 'barracks_status_radiant', 'barracks_status_dire'], axis=1)
y = features['radiant_win']
features = features.drop(['radiant_win'], axis=1)
features = features.fillna(0)

# N — количество различных героев в выборке
N = 112
X_pick = np.zeros((features.shape[0], N))

for i, match_id in enumerate(features.index):
    for p in range(5):
        X_pick[i, features.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, features.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1

/home/worker/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  del sys.path[0]
/home/worker/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


In [56]:
bag = np.concatenate((features.to_numpy(), X_pick), axis=1)
bag.shape

(97230, 214)

In [57]:
clf = LogisticRegression(penalty='l2')
cv = KFold(n_splits=5, shuffle=True)
grid = {
    'C': np.arange(.5, 10.5, .5),
}
gs3 = GridSearchCV(clf, grid, scoring='roc_auc', cv=cv, n_jobs=-1)

In [58]:
start_time3 = datetime.datetime.now()
gs3.fit(bag, y)
end_time3 = datetime.datetime.now()

/home/worker/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/home/worker/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [60]:
gs3.best_estimator_

LogisticRegression(C=0.5, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [61]:
for k, v in gs3.cv_results_.items():
    if 'mean' in k:
        print(f"Key: {k}, value: {v}\n")

Key: mean_fit_time, value: [0.92019582 0.92848353 0.89094281 0.86107149 0.92662916 0.92365222
 0.93468242 0.88968215 0.89804053 0.92649126 0.92503948 0.89343901
 0.89835434 0.92940946 0.93416042 0.93852916 0.92874694 0.92633834
 0.92989292 0.92547388]

Key: mean_score_time, value: [0.06086779 0.02337251 0.03241744 0.02527957 0.02344766 0.02285023
 0.02093191 0.0194737  0.03209643 0.02180037 0.02149315 0.03205228
 0.02160673 0.02316866 0.02151718 0.02207441 0.02259088 0.02281408
 0.02238564 0.02068639]

Key: mean_test_score, value: [0.51345746 0.51345746 0.51345746 0.51345746 0.51345746 0.51345746
 0.51345746 0.51345746 0.51345746 0.51345746 0.51345746 0.51345746
 0.51345746 0.51345746 0.51345746 0.51345746 0.51345746 0.51345746
 0.51345746 0.51345746]

Key: mean_train_score, value: [0.5134527 0.5134527 0.5134527 0.5134527 0.5134527 0.5134527 0.5134527
 0.5134527 0.5134527 0.5134527 0.5134527 0.5134527 0.5134527 0.5134527
 0.5134527 0.5134527 0.5134527 0.5134527 0.5134527 0.5134527]

